In [1]:
import pandas as pd
import numpy as  np
import matplotlib.pyplot as plt
import os
from sklearn.ensemble import RandomForestRegressor 

In [2]:
os.listdir()

['.ipynb_checkpoints',
 'as.ipynb',
 'Bulldozer_Sales_Analysis.ipynb',
 'Datasets',
 'Dipin_0102_Simple_Linear_Regression.ipynb',
 'Dipin_Medical Insurance.ipynb',
 'Fuel_Consumption.ipynb',
 'House_Price_Prediction.ipynb',
 'Notes_02_Visuals.ipynb',
 'Operating_Outliers.ipynb',
 'Patient Expense Prediction Analysis for Doctors.ipynb',
 'Patient Expense Prediction Analysis for Doctors_old.ipynb',
 'Patient-Analysis-Expenses.ipynb',
 'Price_Prediction_Bulldozer.ipynb',
 'Untitled.ipynb',
 'Untitled1.ipynb',
 'Untitled2.ipynb',
 'Untitled3.ipynb']

In [3]:
data_row = pd.read_csv('Datasets/Bulldozer Price_Data.csv', parse_dates=["saledate"])

c:\users\dipin\appdata\local\programs\python\python37-32\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (8,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data_row.describe()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter
count,2.990000e+02,299.000000,2.990000e+02,299.000000,299.0,299.0,299.000000,299.000000
mean,1.139732e+06,32450.000000,8.404672e+05,7380.515050,121.0,3.0,1876.193980,3751.046823
std,2.833651e+02,21949.321205,3.216085e+05,7155.000657,0.0,0.0,329.867101,4624.090975
min,1.139246e+06,5500.000000,4.527000e+03,77.000000,121.0,3.0,1000.000000,0.000000
25%,1.139494e+06,15750.000000,7.149545e+05,3217.000000,121.0,3.0,1996.000000,762.000000
50%,1.139713e+06,27000.000000,1.013386e+06,4606.000000,121.0,3.0,2000.000000,2300.000000
75%,1.139973e+06,41000.000000,1.043462e+06,9535.000000,121.0,3.0,2004.000000,5389.500000
max,1.140232e+06,127000.000000,1.069472e+06,36009.000000,121.0,3.0,2008.000000,29374.000000


In [5]:
data_row.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401123 entries, 0 to 401122
Data columns (total 53 columns):
SalesID                     299 non-null float64
SalePrice                   299 non-null float64
MachineID                   299 non-null float64
ModelID                     299 non-null float64
datasource                  299 non-null float64
auctioneerID                299 non-null float64
YearMade                    299 non-null float64
MachineHoursCurrentMeter    299 non-null float64
UsageBand                   259 non-null object
saledate                    299 non-null datetime64[ns]
fiModelDesc                 299 non-null object
fiBaseModel                 299 non-null object
fiSecondaryDesc             204 non-null object
fiModelSeries               39 non-null object
fiModelDescriptor           72 non-null object
ProductSize                 146 non-null object
fiProductClassDesc          299 non-null object
state                       299 non-null object
ProductGrou

**We need to classify the data based on year only, Lets add a sale year column to data**

In [6]:
data_row['saleyear']=data_row['saledate'].dt.year
data_row.drop('saledate', axis=1, inplace=True)

**We have Object Values and to be converted to Float**

In [7]:
def assign(data,column):
    List_cat=[]
    for i in data[column].unique():
        List_cat.append(i)
    try:
        List_cat.remove(np.nan)
    except Exception as e:
        pass
    
    data[column+'_new']=pd.Categorical(data[column],categories=List_cat,ordered=True).codes
    data.drop(f'{column}', axis=1, inplace=True)
    data.replace(-1, np.nan, inplace=True)

In [8]:
for i in data_row.columns:
    if data_row[i].dtype == 'O':
        try:
            assign(data_row,i)
        except Exception as e:
            pass

In [9]:
data_row.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401123 entries, 0 to 401122
Data columns (total 53 columns):
SalesID                        299 non-null float64
SalePrice                      299 non-null float64
MachineID                      299 non-null float64
ModelID                        299 non-null float64
datasource                     299 non-null float64
auctioneerID                   299 non-null float64
YearMade                       299 non-null float64
MachineHoursCurrentMeter       299 non-null float64
saleyear                       299 non-null float64
UsageBand_new                  259 non-null float64
fiModelDesc_new                299 non-null float64
fiBaseModel_new                299 non-null float64
fiSecondaryDesc_new            204 non-null float64
fiModelSeries_new              39 non-null float64
fiModelDescriptor_new          72 non-null float64
ProductSize_new                146 non-null float64
fiProductClassDesc_new         299 non-null float64
state_n

In [10]:
data_row.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,saleyear,UsageBand_new,...,Undercarriage_Pad_Width_new,Stick_Length_new,Thumb_new,Pattern_Changer_new,Grouser_Type_new,Backhoe_Mounting_new,Blade_Type_new,Travel_Controls_new,Differential_Type_new,Steering_Controls_new
0,1139246.0,66000.0,999089.0,3157.0,121.0,3.0,2004.0,68.0,2006.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
1,1139248.0,57000.0,117657.0,77.0,121.0,3.0,1996.0,4640.0,2004.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2,1139249.0,10000.0,434808.0,7009.0,121.0,3.0,2001.0,2838.0,2004.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251.0,38500.0,1026470.0,332.0,121.0,3.0,2001.0,3486.0,2011.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253.0,11000.0,1057373.0,17311.0,121.0,3.0,2007.0,722.0,2009.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Filling all the empty values with median value of the column**

In [11]:
for i in data_row.columns:
    data_row[i]=data_row[i].fillna(data_row[i].median())

**Seperating input value and output value**

In [12]:
x=data_row.drop('SalePrice', axis=1)
y=np.log(data_row.SalePrice)

**Applying Randon Forest Model**

In [13]:
forr=RandomForestRegressor(n_jobs=1)
forr.fit(x,y)
forr.score(x,y)

0.9418131023617068